# A brief summary of *OSEK OS*
<br>
<div style="opacity: 0.8; font-family: Consolas, Monaco, Lucida Console, Liberation Mono, DejaVu Sans Mono, Bitstream Vera Sans Mono, Courier New; font-size: 12px; font-style: italic;">
    ────────
    for more from the author, visit
    <a href="https://github.com/hazemanwer2000">github.com/hazemanwer2000</a>.
    ────────
</div>

## Table of Contents
* [Data Types](#data-types)
* [System Calls](#system-calls)
  * [Task Management](#task-management)
    * [`ActivateTask`](#activatetask)
    * [`TerminateTask`](#terminatetask)
    * [`ChainTask`](#chaintask)
    * [`Schedule`](#schedule)
    * [`GetTaskID`](#gettaskid)
    * [`GetTaskState`](#gettaskstate)
  * [Interrupt Handling](#interrupt-handling)
  * [Resource Management](#resource-management)
  * [Event Handling](#event-handling)
    * [`SetEvent`](#setevent)
    * [`ClearEvent`](#clearevent)
    * [`GetEvent`](#getevent)
    * [`WaitEvent`](#waitevent)
  * [Alarms](#alarms)
    * [`GetAlarmBase`](#getalarmbase)
    * [`GetAlarm`](#getalarm)
    * [`SetRelAlarm`](#setrelalarm)
    * [`CancelAlarm`](#cancelalarm)
  * [Execution Control](#execution-control)
    * [`GetActiveApplicationMode`](#getactiveapplicationmode)
    * [`StartOS`](#startos)
    * [`ShutdownOS`](#shutdownos)
  * [Hook Routines](#hook-routines)
    * [`ErrorHook(StatusType status)`](#errorhook-statustype-status)
    * [`PreTaskHook(void)`](#pretaskhook-void)
    * [`PostTaskHook(void)`](#posttaskhook-void)
    * [`StartupHook(void)`](#startuphook-void)
    * [`ShutdownHook(void)`](#shutdownhook-void)
  * [Summary](#summary)
* [Specification](#specification)
  * [Task Management](#task-management)
    * [Task Types](#task-types)
  * [Interrupt Handling](#interrupt-handling)
  * [Resource Management](#resource-management)
    * [*OSEK Priority Ceiling Protocol*](#osek-priority-ceiling-protocol)
    * [Internal Resources](#internal-resources)
  * [Alarms](#alarms)
* [Miscellaneous](#miscellaneous)
  * [Conformance Classes](#conformance-classes)
  * [Status Mode](#status-mode)


*OSEK (Offene Systeme und deren Schnittstellen für die Elektronik in Kraftfahrzeugen)* is a standards body that has produced specifications for an embedded operating system, a communication stack, and a network management protocol for automative embedded systems.

For functional safety reasons, all configurations, unless otherwise stated, of the *OSEK OS* (i.e, number of tasks, priorities, etc) are statically configured, and are not configurable during run-time.

## Data Types <a class="anchor" id="data-types"></a>

All data types defined by the *OSEK OS* are of the following form: `UNQIUE-NAME-Type`.

All pointer data types defined by the *OSEK OS* are of the following form: `UNIQUE-NAME-RefType`.

| | |
| --- | --- |
| *Name* | `StatusType` |
| *Description* | This type is returned by system calls. <br> In an ANSI C implementation, it would be an enumeration type.

*Note:* `StatusType` is always `E_OK` if the system call was successful.

| | |
| --- | --- |
| *Name* | `TaskType` |
| *Description* | This type identifies a task within the system.

| | |
| --- | --- |
| *Name* | `TaskStateType` |
| *Description* | This is the type of a task state within the system.

| | |
| --- | --- |
| *Name* | `ResourceType` |
| *Description* | This type identifies a resource within the system.

| | |
| --- | --- |
| *Name* | `EventMaskType` |
| *Description* | This type refers to an event mask.

| | |
| --- | --- |
| *Name* | `TickType` |
| *Description* | This type represents counter values, called *ticks*.

| | |
| --- | --- |
| *Name* | `AlarmBaseType` |
| *Description* | This type is a structure which stores counter characteristics.
| *Elements* | `maxallowedvalue`: Maximum number of ticks. <br> `ticksperbase`: Number of ticks per a counter-specific unit.

| | |
| --- | --- |
| *Name* | `AppModeType` |
| *Description* | This type refers to an application mode.

| | |
| --- | --- |
| *Name* | `OSServiceIdType` |
| *Description* | This type refers to an ID of a system service.

*Note:* All service identifiers can be accessed using the following format: `OSServiceId_SERVICE-NAME`.

## System Calls <a class="anchor" id="system-calls"></a>

Unless otherwise stated, all system calls return a value of `StatusType`.

### Task Management <a class="anchor" id="task-management"></a>

#### `ActivateTask` <a class="anchor" id="activatetask"></a>

| | |
| --- | --- |
| *API* | `ActivateTask(TaskType id)` |
| *Description* | The task is transferred from the *suspended* state into the *ready* state. 

*Note:* For `ActivateTask(..)`, if the number of pending activations exceeds the configured limit, per task, `ES_OS_LIMIT` is returned.

*Note:* When a task transitions from the *suspended* state to the *ready* state, all of its events are cleared.

#### `TerminateTask` <a class="anchor" id="terminatetask"></a>

| | |
| --- | --- |
| *API* | `TerminateTask()` |
| *Description* | The calling task is transferred from the *running* state into the *suspended* state.

#### `ChainTask` <a class="anchor" id="chaintask"></a>

| | |
| --- | --- |
| *API* | `ChainTask(TaskType id)` |
| *Description* | This API combines the behavior of `TerminateTask(..)` and `ActivateTask(..)`.

*Note:* Either `TerminateTask(..)` or `ChainTask(..)` must be called at the end of a task.

#### `Schedule` <a class="anchor" id="schedule"></a>

| | |
| --- | --- |
| *API* | `Schedule()` |
| *Description* | If a higher priority task is *ready*, the internal resource of the task is released, <br> the current task is put in the *ready* state, and the higher priority task in the *running* state.

*Note:* For a task with no internal resource configured (i.e, preemptable), `Schedule(..)` does nothing, naturally.

*Note:* For tasks within a group (i.e, tasks that share an internal resource), `Schedule(..)` may be used by a lower priority task to hand over the processor to a higher priority task.

*Note:* For non-preemptable tasks (i.e, tasks with an internal resource of equal ceiling-priority to `RES_SCHEDULER`, which is the scheduler as a resource, `Schedule(..)` may be used to hand over the processor to a higher priority task.

*Note:* In summary, `Schedule(..)` allows a lower priority task to hand over the processor to a higher priority task, that is unable to pre-empt the lower priority task due to its internal resource ceiling-priority being higher.

#### `GetTaskID` <a class="anchor" id="gettaskid"></a>

| | |
| --- | --- |
| *API* | `GetTaskID(TaskRefType idPtr)` |
| *Description* | This API is used generally used by hook routines to get the `TaskType` of the task in the *running* state. <br> If none, `INVALID_TASK` is returned.

#### `GetTaskState` <a class="anchor" id="gettaskstate"></a>

| | |
| --- | --- |
| *API* | `GetTaskState(TaskType id, TaskStateRefType statePtr)` |
| *Description* | This API is used generally used by hook routines to get the `TaskStateType` of a specific task. <br> Possible states are `RUNNING`, `WAITING`, `READY`, `SUSPENDED`.

### Interrupt Handling <a class="anchor" id="interrupt-handling"></a>

| *API(s)* | *Nested Calls?* | *Description* |
| :-- | :-- | :-- |
| `EnableAllInterrupts` <br> `DisableAllInterrupts` | No | This API enables/disables all interrupts.
| `SuspendAllInterrupts` <br> `ResumeAllInterrupts` | Yes | This API enables/disables all interrupts.
| `SuspendOSInterrupts` <br> `ResumeOSInterrupts` | Yes | This API enables/disables all ISR Category-2 interrupts.

*Note:* Within any critical section established by any interrupt handling API(s), any other type of system call within this critical section is not permitted.

### Resource Management <a class="anchor" id="resource-management"></a>

| *API(s)* | *Description* |
| :-- | :-- |
| `GetResource` <br> `ReleaseResource` | This API permits the aquisition/release of a resource, <br> according to the *OSEK Priority Ceiling Protocol*.

*Note:* Within a critical section established using `GetResource(..)` and `ReleaseResource(..)`, calls to `Schedule(..)`, `WaitEvent(..)`, `TerminateTask(..)`, and `ChainTask(..)` are prohibited.

*Note:* Acquiring and releasing `RES_SCHEDULER` allows the establishing of a critical section within a task, in relation to other tasks only.

### Event Handling <a class="anchor" id="event-handling"></a>

#### `SetEvent` <a class="anchor" id="setevent"></a>

| | |
| --- | --- |
| *API* | `SetEvent(TaskType id, EventMaskType mask)` |
| *Description* | Set the events, specified in `mask`, belonging to task with `TaskType` `id`.

*Note:* It is prohibited to `SetEvent(...)` of a task in the *suspended* state.

#### `ClearEvent` <a class="anchor" id="clearevent"></a>

| | |
| --- | --- |
| *API* | `ClearEvent(EventMaskType mask)` |
| *Description* | Clear the events, specified in `mask`, belonging to the calling task.

#### `GetEvent` <a class="anchor" id="getevent"></a>

| | |
| --- | --- |
| *API* | `SetEvent(TaskType id, EventMaskRefType maskPtr)` |
| *Description* | Set the events, specified in `mask`, belonging to task with `TaskType` `id`.

*Note:* It is prohibited to `GetEvent(...)` of a task in the *suspended* state.

#### `WaitEvent` <a class="anchor" id="waitevent"></a>

| | |
| --- | --- |
| *API* | `WaitEvent(EventMaskType mask)` |
| *Description* | Wait on events, specified in `mask`.

### Alarms <a class="anchor" id="alarms"></a>

For all alarms, `AlarmBaseType` configurations may be accessed as follows:
* `OSMAXALLOWEDVALUE_UNIQUE-NAME`
* `OSTICKSPERBASE_UNIQUE-NAME`

*Note:* The *system counter*, which must exist for an implementation to be *OSEK-compliant*, is accessed similarly, but without a suffix.

#### `GetAlarmBase` <a class="anchor" id="getalarmbase"></a>

| | |
| --- | --- |
| *API* | `GetAlarmBase(AlarmType id, AlarmBaseRefType baseIdPtr)` |
| *Description* | Get the configured `AlarmBaseType` of an alarm.

#### `GetAlarm` <a class="anchor" id="getalarm"></a>

| | |
| --- | --- |
| *API* | `GetAlarm(AlarmType id, TickRefType tickPtr)` |
| *Description* | Get the relative number of ticks before an alarm expires.

*Note:* If the alarm is not in use, `GetAlarm(..)` returns `ES_OS_NOFUNC`.

#### `SetRelAlarm` <a class="anchor" id="setrelalarm"></a>

| | |
| --- | --- |
| *API* | `SetRelAlarm(AlarmType id, TickType initial, TickType cycle)` |
| *Description* | Set an alarm to expire, initially after `initial` number of ticks, and after `cycle` number of ticks thereafter. <br> If `cycle` is `0`, the alarm is not re-set upon expiry.

*Note:* `SetAbsAlarm(..)` is similar to `SetRelAlarm(..)`, except the `initial` number of ticks are absolute, and not relative.

*Note:* If the alarm is in use, `Set-NAME-Alarm(..)` returns `ES_OS_STATE`.

#### `CancelAlarm` <a class="anchor" id="cancelalarm"></a>

| | |
| --- | --- |
| *API* | `CancelAlarm(AlarmType id)` |
| *Description* | Cancel a set alarm.

*Note:* If the alarm is not in use, `CancelAlarm(..)` returns `ES_OS_NOFUNC`.

### Execution Control <a class="anchor" id="execution-control"></a>

#### `GetActiveApplicationMode` <a class="anchor" id="getactiveapplicationmode"></a>

| | |
| --- | --- |
| *API* | `AppModeType GetActiveApplicationMode()` |
| *Description* | Get the active application mode. <br> It is used by user-defined code to tailor program execution accordingly.

*Note:* To be *OSEK-compliant*, the implementation must support the default mode `OSDEFAULTAPPMODE`.

#### `StartOS` <a class="anchor" id="startos"></a>

| | |
| --- | --- |
| *API* | `void StartOS(AppModeType mode)` |
| *Description* | Start the system, passing an `AppModeType`.

*Note:* Upon reset, and before calling `StartOS(..)`, non-portable code determines which application mode to run in, according to, for example, the configuration of a number of hardware pins.

*Note:* `StartOS` may only be called once during run-time, hence application mode may not change during run-time.

#### `ShutdownOS` <a class="anchor" id="shutdownos"></a>

| | |
| --- | --- |
| *API* | `void ShutdownOS(StatusType status)` |
| *Description* | Shutdown a system, passing an error (i.e, not `E_OK`).

### Hook Routines <a class="anchor" id="hook-routines"></a>

#### `ErrorHook(StatusType status)` <a class="anchor" id="errorhook-statustype-status"></a>

| | |
| --- | --- |
| *API* | `void ErrorHook(StatusType status)` |
| *Description* | A hook routine, called whenever a system call returns a status other than `E_OK`, before the call returns.

*Note:* The `OSErrorGetServiceId` macro returns the `OSServiceIdType` of the service where the error has risen.

#### `PreTaskHook(void)` <a class="anchor" id="pretaskhook-void"></a>

| | |
| --- | --- |
| *API* | `void PreTaskHook(void)` |
| *Description* | A hook routine, called after transitioning a task to the *running* state, but before it executes.

#### `PostTaskHook(void)` <a class="anchor" id="posttaskhook-void"></a>

| | |
| --- | --- |
| *API* | `void PostTaskHook(void)` |
| *Description* | A hook routine, called before transitioning an executing task out of the *running* state, but after it stopped execution.

#### `StartupHook(void)` <a class="anchor" id="startuphook-void"></a>

| | |
| --- | --- |
| *API* | `void StartupHook(void)` |
| *Description* | A hook routine, called after the system has been initialized (implementation-specific), but before the scheduler is executing.

#### `ShutdownHook(void)` <a class="anchor" id="shutdownhook-void"></a>

| | |
| --- | --- |
| *API* | `void StartupHook(void)` |
| *Description* | A hook routine, called after the system has ran into a *fatal error*, or `ShutdownOS(..)` has been called.

### Summary <a class="anchor" id="summary"></a>

The figure below shows which contexts are allowed to make which system calls.

<img src="../../.img/osek-api-call-origin.png" width="450" />

## Specification <a class="anchor" id="specification"></a>

### Task Management <a class="anchor" id="task-management"></a>

Each task is configured a priority, and more than one task may be assigned the same priority.

*Note:* Ready tasks of the same priority enter a queue, and execute in the order *oldest-to-newest*.

*Note:* When no tasks are ready to execute, the system becomes *idle*.

#### Task Types <a class="anchor" id="task-types"></a>

*OSEK* defines two types of tasks, *basic* and *extended*.
* Extended tasks may wait on events, while basic tasks may not.
* Basic tasks may be activated more than once, while extended tasks may be activated at most once.

*Note:* Events are configured, and mapped `1:1` per extended task.

*Note:* Usually, in an implementation, extended tasks demand more resource usage.

The figure below shows the state diagram of basic tasks.

<img src="../../.img/osek-task-state-diagram-basic.png" width="250" />

The figure below shows the state diagram of extended tasks.

<img src="../../.img/osek-task-state-diagram-extended.png" width="375" />

### Interrupt Handling <a class="anchor" id="interrupt-handling"></a>

*OSEK* defines two types of interrupts:
* ISR Category-1, may not make any (except interrupt handling) system calls.
* ISR Category-2, may make more system calls.

### Resource Management <a class="anchor" id="resource-management"></a>

A resource must be acquired and released in a *Last-In-First-Out (LIFO)* order.

#### *OSEK Priority Ceiling Protocol* <a class="anchor" id="osek-priority-ceiling-protocol"></a>

To avoid possible priority inversion and deadlocks, *OSEK* defines the *OSEK Priority Ceiling Protocol*.
* Each task specifies which resources it uses.
* Each resource is assigned a *ceiling priority*, which is the highest priority of any task that uses that resource.
* When a task acquires a resource, it inherits the ceiling-priority of that resource.
* Before releasing the resource, the task may not call `TerminateTask(..)` or `ChainTask(..)`, and if the task is extended, it may not call `WaitEvent(..)`, which gurantees it will not transition out of either the *running*, or *ready* states.

*Note:* The *OSEK Priority Ceiling Protocol* may be extended to interrupts, but not necessary for an implementation to be *OSEK-compliant*.

#### Internal Resources <a class="anchor" id="internal-resources"></a>

Each task may be configured an *internal resource*, that:
* It inherits the ceiling-priority of (i.e, acquires it) when entering into the *running* state,
* And returns to its configured priority (i.e, releases its internal resource) when leaving the *running* state.

Hence,
* Tasks with a shared internal resource may not preempt each other, and constitute a *task group*.
* Fully preemptable tasks have no internal resource configured.
* Non-preemptable tasks have *RES_SCHEDULER* as an internal resource configured, which has the highest ceiling-priority in the system.

*Note:* Notice that when an interrupt preempts an executing task, it does not leave the *running* state, and hence, it does not release its internal resource.

### Alarms <a class="anchor" id="alarms"></a>

An expiring alarm may be configured to perform different actions:
* Activate a suspended task.
* Trigger event(s).
* Execute a callback routine.

## Miscellaneous <a class="anchor" id="miscellaneous"></a>

### Conformance Classes <a class="anchor" id="conformance-classes"></a>

*OSEK* defines a number of conformance classes, that implementations may adhere to.

<img src="../../.img/osek-conformance-classes.png" width="475" />

### Status Mode <a class="anchor" id="status-mode"></a>

*OSEK* defines two status modes, system-wide, *standard* and *extended*.

In *extended* mode, more fatal errors (i.e, errors that induce a forced-system shutdown) reported as application errors. It is typically employed in the testing phase.

*Standard* mode is employed in production ECUs, to minimize unnecessary resource overhead.